In [34]:
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

1

In [35]:
data = pd.read_csv('customer_churn.csv')

In [36]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


2, 3 & 4

In [37]:
model_data_x = pd.DataFrame()
model_data_x['tenure'] = data['tenure']
model_data_x['SeniorCitizen'] = data['SeniorCitizen']
model_data_x['MonthlyCharges'] = data['MonthlyCharges']

In [38]:
model_data_x

,tenure,SeniorCitizen,MonthlyCharges
0,1,0,29.85
1,34,0,56.95
2,2,0,53.85
3,45,0,42.30
4,2,0,70.70
...,...,...,...
7038,24,0,84.80
7039,72,0,103.20
7040,11,0,29.60
7041,4,1,74.40


In [39]:
model_data_y = data['Churn']

In [40]:
model_data_y

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

In [41]:
transformer = Normalizer().fit(model_data_x)
x_normalized = transformer.transform(model_data_x)
x = pd.DataFrame(x_normalized)
x.columns = model_data_x.columns
x.head()

,tenure,SeniorCitizen,MonthlyCharges
0,0.033482,0.0,0.999439
1,0.512610,0.0,0.858622
2,0.037115,0.0,0.999311
3,0.728628,0.0,0.684910
4,0.028277,0.0,0.999600


5 & 6

In [42]:
X_train, X_test, y_train, y_test = train_test_split(model_data_x, model_data_y, test_size=0.4)

In [43]:
classification = LogisticRegression().fit(X_train, y_train)

In [44]:
predictions = classification.predict(X_test)
predictions

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [45]:
classification.score(X_test, y_test)

0.7927608232789212

In [46]:
y_test.value_counts()

No     2084
Yes     734
Name: Churn, dtype: int64

In [47]:
pd.Series(predictions).value_counts()

No     2316
Yes     502
dtype: int64

7

This occurs because since the dataset we used to train the model has such a high descrepancy between "Yes" and "No"(high occurance of no's), the model with predict a lot of "no's", when he should be predicting "yes"

8

In [48]:
smote = SMOTE()
X = model_data_x
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = model_data_y
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

No     5174
Yes    5174
Name: Churn, dtype: int64

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.4)

In [50]:
classification = LogisticRegression().fit(X_train, y_train)

In [51]:
predictions = classification.predict(X_test)
predictions

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype=object)

In [52]:
classification.score(X_test, y_test)

0.7301932367149758

In [53]:
y_test.value_counts()

No     2107
Yes    2033
Name: Churn, dtype: int64

In [54]:
pd.Series(predictions).value_counts()

Yes    2106
No     2034
dtype: int64

Yes, there is. Even though the accuracy hasn't changed much, in fact it is slightly lower, the model has now detected a lot more true positives which is what we needed.